Why the choice of preprocessing strategies matters: Botvinik-Nezer, R., Holzmeister, F., Camerer, C.F. et al. Variability in the analysis of a single neuroimaging dataset by many teams. Nature (2020). https://doi.org/10.1038/s41586-020-2314-9. From the abstract: "The flexibility of analytical approaches is exemplified by the fact that no two teams chose identical workflows to analyse the data. This flexibility resulted in sizeable variation in the results of hypothesis tests, even for teams whose statistical maps were highly correlated at intermediate stages of the analysis pipeline....Our findings show that analytical flexibility can have substantial effects on scientific conclusions, and identify factors that may be related to variability in the analysis of functional magnetic resonance imaging. The results emphasize the importance of validating and sharing complex analysis workflows, and demonstrate the need for performing and reporting multiple analyses of the same data."

# fMRIPrep
I know I said I wasn't going to cover fmriprep last week, but I changed my mind. Here's how to do it, but after I cover this we'll dive into each step more deeply.  
What is fMRIPrep? "fMRIPrep is a functional magnetic resonance imaging (fMRI) data preprocessing pipeline that is designed to provide an easily accessible, state-of-the-art interface that is robust to variations in scan acquisition protocols and that requires minimal user input." It's a big black box that takes a bids dataset and spits out preprocessed data.  
fMRIprep's web site is here: https://fmriprep.readthedocs.io/en/stable/index.html  
# Installing fMRIprep
There are multiple ways to install fMRIprep, but since we are on a HPC cluster we're going to run it in a singularity container. Singularity is installed on talapas, but you'll need to build the fMRIprep container yourself. That will look like this:    
```
module load singularity
singularity build PATHNAME/fmriprep-VERSION.simg docker://poldracklab/fmriprep:VERSION
```
We need to define two thing: the path where you are going to store the container, and the version of fMRIprep we wish to install. According to https://fmriprep.readthedocs.io/en/stable/changes.html the latest version is 20.0.7. For the path, choose something under your pirg project directory. There won't be room for it under your home directory on talapas.

In [1]:
mypirg = 'lcni' # change this to your own pirg

In [2]:
import pathlib
pirg_path = pathlib.Path().home() / mypirg

In [3]:
# make sure you did that right
pirg_path.exists()

True

In [4]:
# make a directory for singularity images, if you don't have one already
image_path = pirg_path / 'sing_images'

In [5]:
image_path.mkdir(exist_ok = True) # create the directory, don't give an error if it already exists

In [6]:
# define which version we'll build
version = '20.0.7'

Quick reminder of how string formating works in python

In [7]:
'directory is {0}, version is {1}'.format(image_path, version)

'directory is /home/jolinda/lcni/sing_images, version is 20.0.7'

In [8]:
# our build command is
build_command = 'singularity build {0}/fmriprep-{1}.simg docker://poldracklab/fmriprep:{1}'.format(image_path, version)

Building will take a while, let's use slurmpy.

In [9]:
import slurmpy

In [10]:
job = slurmpy.SlurmJob()
job.account = mypirg
job.jobname = 'build_fmriprep'

# remember, slurmpy takes the commands as a list of strings
job.command = list()
job.command.append('module load singularity')
job.command.append(build_command)

job.WriteSlurmFile()
job.PrintSlurmFile()

#!/bin/bash
#SBATCH --job-name=build_fmriprep
#SBATCH --account=lcni

module load singularity
singularity build /home/jolinda/lcni/sing_images/fmriprep-20.0.7.simg docker://poldracklab/fmriprep:20.0.7


In [32]:
job.SubmitSlurmFile()

Submitted batch job 11949409



'11949409'

THIRTY-SEVEN MINUTES LATER

In [34]:
job.ShowStatus()

COMPLETED 1


In [35]:
job.ShowOutput()

slurm-11949409.out
INFO:    Starting build...
Getting image source signatures
Skipping fetch of repeat blob sha256:0a01a72a686c389637334de1e2d0012da298960366f6d8f358b8e10dc3b5e330
Skipping fetch of repeat blob sha256:cc899a5544da1a6cfb970d2484d32c063f8df26a430d92f39c98e72261e226f2
Skipping fetch of repeat blob sha256:19197c55075519928dd2ff059745665a2c9b72f4e8af6f7a1ce662e696d339bd
Skipping fetch of repeat blob sha256:716d454e56b61d1343a01f3b1829574333e2e3df20e77d1958d7b0b939ea1b61
Skipping fetch of repeat blob sha256:b5bf898e214a893171c1e1ab287fc7f1d3573e414869f21f06e3468fea43add3
Skipping fetch of repeat blob sha256:42da0942cc0e3de8b86ba649f506f3a0c87533451756c51d7b8bf181ba94a2eb
Skipping fetch of repeat blob sha256:14f5757104e98f3e93ab8930b4afd4eb21146c829f97cb5f942f31f67419dfc1
Skipping fetch of repeat blob sha256:611fe4f705a58c51fd740b790122f3f5900386b72f3a4c5bed2f8bd6b5c28a19
Skipping fetch of repeat blob sha256:ac0b78389510ddc5f05baf66cce3660bd59dcc6c40f868b039805e42f61755fb
Skip

In [36]:
print(job.JobInfo())

               JobID                   JobName  Partition      State    Elapsed     MaxRSS 
-------------------- ------------------------- ---------- ---------- ---------- ---------- 
            11949409            build_fmriprep      short  COMPLETED   00:36:47            
      11949409.batch                     batch             COMPLETED   00:36:47    302964K 
     11949409.extern                    extern             COMPLETED   00:36:47          0 



# Testing the installation

In [11]:
fmriprep = image_path / 'fmriprep-{}.simg'.format(version)

In [12]:
fmriprep.exists()

True

To test this, you can go into a talapas shell and run the following commands:
```
module load singularity
singularity shell --cleanenv {path to fmriprep image}
```
This will open a singularity shell with a clean environment. You'll be inside the current working directory. Exit by typing 'exit'.  
We can also check it from within the notebook using "singularity exec {bash command}". 

In [13]:
%%bash -s {str(fmriprep)} 
module load singularity 
singularity exec --cleanenv $1 ls *.ipynb

2.PythonAndJupyterNotebook.ipynb
7.ExaminingData.ipynb
8.fmriprep.ipynb
CopyTheseCommands.ipynb
Untitled.ipynb
Untitled1.ipynb
Untitled2.ipynb
Untitled3.ipynb
Untitled4.ipynb
Untitled5.ipynb
intro_to_python_teaching.ipynb
mynotebook.ipynb


# Running fmriprep with singularity
To run fmriprep with a single subject, we'll use this command:
```
singularity run --cleanenv {bidsdir} {outputdir} participant --participant-label {label}
```
'label' is your subject label NOT include sub-. Let's define our bids and output directories.

In [14]:
bidsdir = pathlib.Path('/projects/lcni/jolinda/shared/TalapasClass/ds000114/')

In [15]:
outputdir = pirg_path / 'fmriprep'

In [16]:
outputdir.mkdir(exist_ok = True)

In [17]:
outputdir.exists()

True

What happens if we try to run fmriprep now? Two errors: first, it won't be able to access those directories. How do I know this? Try this (FYI I'm using '--no-raise-error' with the %%bash magic to only show the std error output, not the big long python error we'd normally see)

In [18]:
%%bash -s {str(fmriprep)} {str(bidsdir)} --no-raise-error
module load singularity 
singularity exec --cleanenv $1 ls $2

ls: cannot access '/projects/lcni/jolinda/shared/TalapasClass/ds000114': No such file or directory


That path does not exist in our singularity instance! We need to either define it relative to the current working directory, or use the singularity 'bind' command.

In [19]:
%%bash -s {str(fmriprep)} {str(bidsdir)} --no-raise-error
module load singularity 
singularity exec --cleanenv --bind $2:/bids --cleanenv $1 ls /bids

CHANGES
dataset_description.json
dwi.bval
dwi.bvec
participants.tsv
sub-01
sub-02
sub-03
sub-04
sub-05
sub-06
sub-07
sub-08
sub-09
sub-10
task-covertverbgeneration_bold.json
task-covertverbgeneration_events.tsv
task-fingerfootlips_bold.json
task-fingerfootlips_events.tsv
task-linebisection_bold.json
task-overtverbgeneration_bold.json
task-overtverbgeneration_events.tsv
task-overtwordrepetition_bold.json
task-overtwordrepetition_events.tsv


We'll need to specify the bids directory, the output directory and a work directory. If we don't specify a work directory fmriprep will create one under the current working directory. This will be a problem if you're currently in your home directory, because you'll run out of space. First make a work directory somewhere. I'm putting it in the pirg_dir, which I'll bind. 

In [22]:
workdir = pirg_path / 'work'
workdir.mkdir(exist_ok = True)

We need one last thing: a freesurfer license file. Otherwise fmriprep will fail when it gets to that stage. They are free, you can get one by registering on the freesurfer web site, or there's one in the class directory you can use. It needs to be somewhere fmriprep can find it, for example, in pirg_path. Copying files is a little easier in bash than python:

In [28]:
!cp '/projects/lcni/jolinda/shared/TalapasClass/license.txt' {pirg_path}

We can bind multiple directories by separating them with commas, (no spaces). Let's check that we can find the fmriprep and work directories and the freesurfer license

In [33]:
%%bash -s {str(fmriprep)} {str(bidsdir)} {str(pirg_path)} --no-raise-error
module load singularity 
singularity exec --cleanenv --bind $2:/bids,$3:/pirg --cleanenv $1 ls /pirg

blank
fmriprep
license.txt
shared
sing_images
temp
work


Now our command looks like (using \ to break up long lines):
```
singularity run --cleanenv \
--bind {bidsdir}:/bids,{pirg_path}:/pirg \
/bids /pirg/fmriprep \
participant --participant-label -01 \
--work-dir=/pirg/work \
--fs-license-file=/pirg/license.txt
```

I'm going to add two more options: --nthreads N and --sloppy. Using additional threads will speed things up considerably for things that can take advantage of it and fmriprep is one of those things (I know that freesurfer is multithreaded, I don't know about the rest of it). On talapas there are 28 cores per cpu, so we could use up to 28 threads. I used 8 threads here, feel free to use more. 'sloppy' is good for testing whether your pipeline runs without errors. It'll be faster than the "real" analysis by roughly a factor of two.

In [35]:
nthreads = 8
fmriprep_command = list()
fmriprep_command.append('module load singularity')
fmriprep_command.append('singularity run --cleanenv \\')
fmriprep_command.append('--bind {}:/bids,{}:/pirg {} \\'.format(bidsdir, pirg_path, fmriprep))
fmriprep_command.append('/bids /pirg/fmriprep participant --participant-label 01 \\')
fmriprep_command.append('--work-dir=/pirg/work \\')
fmriprep_command.append('--fs-license-file=/pirg/license.txt \\')
fmriprep_command.append('--sloppy --nthreads={}'.format(nthreads))
[print(x) for x in fmriprep_command]

module load singularity
singularity run --cleanenv \
--bind /projects/lcni/jolinda/shared/TalapasClass/ds000114:/bids,/home/jolinda/lcni:/pirg /home/jolinda/lcni/sing_images/fmriprep-20.0.7.simg \
/bids /pirg/fmriprep participant --participant-label 01 \
--work-dir=/pirg/work \
--fs-license-file=/pirg/license.txt \
--sloppy --nthreads=8


[None, None, None, None, None, None, None]

I know from past runs that fmriprep will take around 5G of ram and, will complete in 4.5 hours with these parameters (remove 'sloppy' and it will take 7.5 hours). So the default short partition is fine, and so is the default memory allocation (a little more than 4G per cpu). Note that if we weren't requesting more threads, we would need to request more memory.

In [36]:
job = slurmpy.SlurmJob()

In [37]:
job.jobname = 'fmriprep'

In [38]:
job.account = mypirg

In [39]:
# when using threads you need to include that in the sbatch parameters
# this sets the 'cpus-per-task' parameter
# We can't say job.cpus-per-task = nthreads because we can't have dashes inside python variable names, 
# so this is one of our "special" parameters
job.threads = nthreads

In [40]:
job.command = fmriprep_command

In [41]:
job.WriteSlurmFile()

'fmriprep.srun'

In [42]:
job.PrintSlurmFile()

#!/bin/bash
#SBATCH --job-name=fmriprep
#SBATCH --cpus-per-task=8
#SBATCH --account=lcni

module load singularity
singularity run --cleanenv \
--bind /projects/lcni/jolinda/shared/TalapasClass/ds000114:/bids,/home/jolinda/lcni:/pirg /home/jolinda/lcni/sing_images/fmriprep-20.0.7.simg \
/bids /pirg/fmriprep participant --participant-label 01 \
--work-dir=/pirg/work \
--fs-license-file=/pirg/license.txt \
--sloppy --nthreads=8


We can submit that with job.SubmitSlurmFile(). What if we want to run it on all of our subjects? We need an array of participant labels. Remember pathlib's built in globbing from last week? This time we'll use glob, not relative glob (rglob), because I want to match just the sub- directories directly under the top level bids-dir:

In [44]:
sorted(bidsdir.glob('sub-*'))

[PosixPath('/projects/lcni/jolinda/shared/TalapasClass/ds000114/sub-01'),
 PosixPath('/projects/lcni/jolinda/shared/TalapasClass/ds000114/sub-02'),
 PosixPath('/projects/lcni/jolinda/shared/TalapasClass/ds000114/sub-03'),
 PosixPath('/projects/lcni/jolinda/shared/TalapasClass/ds000114/sub-04'),
 PosixPath('/projects/lcni/jolinda/shared/TalapasClass/ds000114/sub-05'),
 PosixPath('/projects/lcni/jolinda/shared/TalapasClass/ds000114/sub-06'),
 PosixPath('/projects/lcni/jolinda/shared/TalapasClass/ds000114/sub-07'),
 PosixPath('/projects/lcni/jolinda/shared/TalapasClass/ds000114/sub-08'),
 PosixPath('/projects/lcni/jolinda/shared/TalapasClass/ds000114/sub-09'),
 PosixPath('/projects/lcni/jolinda/shared/TalapasClass/ds000114/sub-10')]

In [45]:
[x.name for x in bidsdir.glob('sub-*')]

['sub-03',
 'sub-08',
 'sub-02',
 'sub-07',
 'sub-01',
 'sub-06',
 'sub-05',
 'sub-04',
 'sub-09',
 'sub-10']

In [46]:
# drop the first four characters: 'sub-', and sort
sorted([x.name[4:] for x in bidsdir.glob('sub-*')])

['01', '02', '03', '04', '05', '06', '07', '08', '09', '10']

Nice, we have a command that should work for ANY bids directory. Let's modify our job to use an array.

In [47]:
job.array = sorted([x.name[4:] for x in bidsdir.glob('sub-*')])

In [166]:
cp '/projects/lcni/jolinda/shared/TalapasClass/license.txt' license.txt

In [48]:
nthreads = 16
fmriprep_command = list()
fmriprep_command.append('module load singularity')
fmriprep_command.append('singularity run --cleanenv \\')
fmriprep_command.append('--bind {}:/bids,{}:/pirg {} \\'.format(bidsdir, pirg_path, fmriprep))
fmriprep_command.append('/bids /pirg/fmriprep participant --participant-label ${x} \\')
fmriprep_command.append('--work-dir=/pirg/work \\')
fmriprep_command.append('--fs-license-file=/pirg/license.txt \\')
fmriprep_command.append('--nthreads={}'.format(nthreads))

job.threads = nthreads
job.command = fmriprep_command
job.jobname = 'fprep_array'

In [49]:
job.WriteSlurmFile()

'fprep_array.srun'

In [50]:
job.PrintSlurmFile()

#!/bin/bash
#SBATCH --job-name=fprep_array
#SBATCH --cpus-per-task=16
#SBATCH --account=lcni
#SBATCH --array=0-9

data=(01 02 03 04 05 06 07 08 09 10)

x=${data[$SLURM_ARRAY_TASK_ID]}


module load singularity
singularity run --cleanenv \
--bind /projects/lcni/jolinda/shared/TalapasClass/ds000114:/bids,/home/jolinda/lcni:/pirg /home/jolinda/lcni/sing_images/fmriprep-20.0.7.simg \
/bids /pirg/fmriprep participant --participant-label ${x} \
--work-dir=/pirg/work \
--fs-license-file=/pirg/license.txt \
--nthreads=16


In [52]:
job.SubmitSlurmFile()

Submitted batch job 11988466



'11988466'

In [55]:
job.ShowStatus()

FAILED 1
RUNNING 9


In [56]:
print(job.JobInfo())

               JobID                   JobName  Partition      State    Elapsed     MaxRSS 
-------------------- ------------------------- ---------- ---------- ---------- ---------- 
          11988466_0               fprep_array      short    RUNNING   00:00:28            
    11988466_0.batch                     batch               RUNNING   00:00:28            
   11988466_0.extern                    extern               RUNNING   00:00:28            
          11988466_1               fprep_array      short    RUNNING   00:00:28            
    11988466_1.batch                     batch               RUNNING   00:00:28            
   11988466_1.extern                    extern               RUNNING   00:00:28            
          11988466_2               fprep_array      short     FAILED   00:00:12            
    11988466_2.batch                     batch                FAILED   00:00:12          0 
   11988466_2.extern                    extern             COMPLETED   00:00:12 

In [57]:
#print that one
job.ShowOutput(2)

slurm-11988466_2.out
bids-validator@1.4.0

	1: [ERR] Files with such naming scheme are not part of BIDS specification. This error is most commonly caused by typos in file names that make them not BIDS compatible. Please consult the specification and make sure your files are named correctly. If this is not a file naming issue (for example when including files not yet covered by the BIDS specification) you should include a ".bidsignore" file in your dataset (see https://github.com/bids-standard/bids-validator#bidsignore for details). Please note that derived (processed) data should be placed in /derivatives folder and source data (such as DICOMS or behavioural logs in proprietary formats) should be placed in the /sourcedata folder. (code: 1 - NOT_INCLUDED)
		./sub-03/ses-test/func/sub-03_ses-test_task-linebisection_bold.nii.gz.crswap
			Evidence: sub-03_ses-test_task-linebisection_bold.nii.gz.crswap

	Please visit https://neurostars.org/search?q=NOT_INCLUDED for existing conversations ab

One of them didn't meet the bids spec. I would have known that if I had run the dataset through the bids validator first. 